# Яременко Вікторія Лабораторна 2

Створити RNN для семантичного аналізу тексту з використанням:
﻿﻿﻿моделі LSTM;

In [3]:
import numpy as np
import re
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Задані тексти
texts_true = [
    "Ця книга вражає своєю глибиною та проникливістю.",
    "Автор зумів майстерно розкрити головних героїв, надаючи їм життєвий досвід.",
    "Читання цієї книги залишає після себе незабутні враження та відчуття вдячності."
]

texts_false = [
    "Ця книга розчарувала своїми плоскими персонажами та передбачуваним сюжетом.",
    "Стиль автора в цій книзі відчувається дуже механічним та бездушним.",
    "Читання цієї книги не принесло жодного задоволення, лише втрату часу та розчарування."
]

# Об'єднання текстів
texts = texts_true + texts_false
count_true = len(texts_true)
count_false = len(texts_false)
total_lines = count_true + count_false
print(count_true, count_false, total_lines)

maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(texts)

dist = list(tokenizer.word_counts.items())
print(dist[:10])
print(texts[0][:100])

max_text_len = 10
data = tokenizer.texts_to_sequences(texts)
data_pad = pad_sequences(data, maxlen=max_text_len)
print(data_pad)

print(list(tokenizer.word_index.items()))

X = data_pad
Y = np.array([[1, 0]] * count_true + [[0, 1]] * count_false)
print(X.shape, Y.shape)

indices = np.random.choice(X.shape[0], size=X.shape[0], replace=False)
X = X[indices]
Y = Y[indices]

model = Sequential()
model.add(Embedding(maxWordsCount, 128, input_length=max_text_len))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(2, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.0001))

history = model.fit(X, Y, batch_size=32, epochs=50)

reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

def sequence_to_text(list_of_indices):
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return words

t = "Я люблю читати книги".lower()
data = tokenizer.texts_to_sequences([t])
data_pad = pad_sequences(data, maxlen=max_text_len)
print(sequence_to_text(data[0]))

res = model.predict(data_pad)
print(res, np.argmax(res), sep='\n')


3 3 6
[('ця', 2), ('книга', 2), ('вражає', 1), ('своєю', 1), ('глибиною', 1), ('та', 5), ('проникливістю', 1), ('автор', 1), ('зумів', 1), ('майстерно', 1)]
Ця книга вражає своєю глибиною та проникливістю.
[[ 0  0  0  2  3  7  8  9  1 10]
 [11 12 13 14 15 16 17 18 19 20]
 [ 5  6 21 22 23 24 25  1 26 27]
 [ 0  2  3 28 29 30 31  1 32 33]
 [34 35 36 37 38 39 40 41  1 42]
 [ 6 43 44 45 46 47 48 49  1 50]]
[('та', 1), ('ця', 2), ('книга', 3), ('читання', 4), ('цієї', 5), ('книги', 6), ('вражає', 7), ('своєю', 8), ('глибиною', 9), ('проникливістю', 10), ('автор', 11), ('зумів', 12), ('майстерно', 13), ('розкрити', 14), ('головних', 15), ('героїв', 16), ('надаючи', 17), ('їм', 18), ('життєвий', 19), ('досвід', 20), ('залишає', 21), ('після', 22), ('себе', 23), ('незабутні', 24), ('враження', 25), ('відчуття', 26), ('вдячності', 27), ('розчарувала', 28), ('своїми', 29), ('плоскими', 30), ('персонажами', 31), ('передбачуваним', 32), ('сюжетом', 33), ('стиль', 34), ('автора', 35), ('в', 36), ('ц

In [5]:
# Визначення результату
if res[0][0] > res[0][1]:
    print("Результат. Текст позитивний.")
else:
    print("Результат. Текст негативний.")


Результат. Текст позитивний.
